# Use this python notebook to analyze, understand, and interpret the results of your work in lab 2a.<br>
1. First of all, we import the needed Python libraries, and read the file _triangular.csv_<br>
If _ternary_ library is not installed in your system, open the "Anaconda Powershell prompt" and type:<br> 
pip install python-ternary<br>
Make sure your data are separated by commas in the csv file. Otherwise, change the delimiter when reading _triangular.csv_

In [ ]:
#lab 2a
# Ternary plots: anti-clockwise
import ternary
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

def quadratic(x,a,b,c):
    return a + b * x + c * x * x

def cubic(x,a,b,c,d):
    return a + b * x + c * x * x + d * x * x * x

#Points

points = []
pointsi = []
xOC = []
xW  = []

masses = pd.read_csv("triangular.csv",sep=',')
print(masses)

2. We now plot the binodal curve in the ternary diagram, interpolating the values.

In [ ]:
mOC  = masses['mOC'].values
mW   = masses['mW'].values
mAcH = masses['mAcH'].values

#mass fractions
xOC  = mOC  / (mOC + mW + mAcH)
xW   = mW   / (mOC + mW + mAcH)
xAcH = mAcH / (mOC + mW + mAcH)
#put the fractions in array points to plot later on
for j in range(len(xOC)):
    points.append([xOC[j],xAcH[j],xW[j]])

degree = int(input('Enter the degree of the polynomial to fit the data (2 or 3)= '))
#fit
rms = 0
if degree == 2:
    best_vals, covar = curve_fit(quadratic, xOC, xW)
# rms of the fit
    for i, value in enumerate(xOC):
        rms += (quadratic(value,*best_vals) - xW[i]) ** 2
elif degree == 3:
    best_vals, covar = curve_fit(cubic, xOC, xW)
# rms of the fit
    for i, value in enumerate(xOC):
        rms += (cubic(value,*best_vals) - xW[i]) ** 2
    
rms = np.sqrt ( rms / len(xOC) )
print("RMS = ",rms)

#denser plot with the fit
xOCi = np.linspace(0, 1, num=100, endpoint=True)
if degree == 2:
    xWi = quadratic(xOCi,*best_vals)
else:
    xWi = cubic(xOCi,*best_vals)
s = xOCi + xWi
n = []
for i,ele in enumerate(s):
    if ele > 1:
        s[i] = 1
    n.append(i)
xAcHi = 1 - s
#put the fitted fractions in an array to plot
for j in range(len(xOCi)):
    point = [xOCi[j],xAcHi[j],xWi[j]]
    pointsi.append(point)

fig, tax = ternary.figure(scale=1.0)
fig.set_size_inches(10, 10)

# Plot the data
#Scatter
tax.scatter(points, marker='o', color='black')
#line
tax.plot(pointsi, linewidth=2.0, color='blue')

fontsize=20

tax.right_corner_label("OC", fontsize=fontsize)
tax.top_corner_label("AcH", fontsize=fontsize)
tax.left_corner_label("W", fontsize=fontsize)

tax.get_axes().axis('off')
tax.clear_matplotlib_ticks()
tax.ticks(axis='lbr', linewidth=1, multiple=0.1, tick_formats="%.1f")
tax.boundary()

#tax.gridlines(multiple=0.1, color="black")
tax.gridlines(color="black", multiple=0.01, linewidth=0.5)
tax.gridlines(color="black", multiple=0.1, linewidth=1)
tax.show()
tax.savefig('ternary.svg')

Assume now, you have a binary mixture AcH-OC and we add W. Enter different values of $x_{\mathrm{OC}}$ to determine the minimum values that can be employed to construct the binodal curve by adding amounts of W.<br>
__Q. Why starting with AcH-OC binary mixtures only the right side or the binodal curve is constructed?__

In [ ]:
def line(xOC,xOC1):
    slope =  - 1 / xOC1
    return slope * (xOC - xOC1)

xOC1 = float(input('Mole fraction of OC in the initial binary mixture = '))

xOC = np.linspace(0, xOC1, num=100, endpoint=True)
xW = line(xOC,xOC1)
xAcH = 1 - xOC - xW

if degree == 2:
    coeff = [ -best_vals[2], -1 -best_vals[1] , 1 - best_vals[0]]
else:
    coeff = [ -best_vals[3], -best_vals[2], -1 - best_vals[1]  , 1 - best_vals[0]]
print('Solubilities or the W-OC binary mixture from the fit')
for ele in np.roots(coeff):
    if ele >= 0 and ele <= 1:
        print(ele)

if degree == 2:
    coeff = [ -best_vals[2], -1 -best_vals[1]  - (1 / xOC1 -1), 1 - best_vals[0]]
else:
    coeff = [ -best_vals[3], -best_vals[2], -1 - best_vals[1] - (1 / xOC1 -1) , 1 - best_vals[0]]
print('Points of the binodal curve crossed by the line')
for ele in np.roots(coeff):
    imag=ele.imag
    if ele >= 0 and ele <= 1 and imag==0:
        print('xOC = ',ele,'xW = ',line(ele,xOC1),'xAcH = ',1-ele-line(ele,xOC1))

points = []
#put the fitted fractions in an array to plot
for j in range(len(xOC)):
    point = [xOC[j],xAcH[j],xW[j]]
    points.append(point)

fig, tax = ternary.figure(scale=1.0)
fig.set_size_inches(10, 10)

# Plot the data
#Scatter
tax.plot(points, linewidth=2.0, color='black')
#line
tax.plot(pointsi, linewidth=2.0, color='blue')

fontsize=20

tax.right_corner_label("OC", fontsize=fontsize)
tax.top_corner_label("AcH", fontsize=fontsize)
tax.left_corner_label("W", fontsize=fontsize)

tax.get_axes().axis('off')
tax.clear_matplotlib_ticks()
tax.ticks(axis='lbr', linewidth=1, multiple=0.1, tick_formats="%.1f")
tax.boundary()

#tax.gridlines(multiple=0.1, color="black")
tax.gridlines(color="black", multiple=0.01, linewidth=0.5)
tax.gridlines(color="black", multiple=0.1, linewidth=1)
tax.show()